In [1]:
import numpy as np
import cv2

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
# We can access the dataset through relative paths as such:
DAY_PATH = "./data/clear_images.npy"
NIGHT_PATH = "./data/night_images.npy"

import numpy as np

day_images = np.load(DAY_PATH)
night_images = np.load(NIGHT_PATH)

print("Day images shape:", day_images.shape)
print("Night images shape:", night_images.shape)

Day images shape: (5000, 224, 224, 3)
Night images shape: (5000, 224, 224, 3)


In [3]:
# Train Test Split

X_images = np.concatenate((day_images, night_images))
y_images = np.array([0]*len(day_images) + [1]*len(night_images))

X_train, X_test, y_train, y_test = train_test_split(X_images, y_images, test_size=0.2, random_state=42)


In [4]:
# Create the model

dropout = 0.35

model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(224, 224, 3)),
tf.keras.layers.MaxPooling2D(2, 2),
tf.keras.layers.Dropout(dropout/4),
tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Dropout(dropout/3),
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Dropout(dropout/2),
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Dropout(dropout),
tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),
tf.keras.layers.Dropout(dropout),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(512, activation='relu'),
tf.keras.layers.Dropout(dropout),
tf.keras.layers.Dense(3, activation='softmax')])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 16)      0         
_________________________________________________________________
dropout (Dropout)            (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 64)        1

In [5]:
# Training

callbacks = [
    tf.keras.callbacks.ModelCheckpoint("model_checkpoint.h5", monitor="val_loss", mode="min", save_best_only=True, verbose=0)
]

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), callbacks=callbacks)

Epoch 1/5
250/250 [==============================] - 39s 134ms/step - loss: 3.0286 - accuracy: 0.8621 - val_loss: 0.2003 - val_accuracy: 0.9690
Epoch 2/5
250/250 [==============================] - 33s 130ms/step - loss: 0.1045 - accuracy: 0.9731 - val_loss: 0.2325 - val_accuracy: 0.9575
Epoch 3/5
250/250 [==============================] - 32s 130ms/step - loss: 0.0968 - accuracy: 0.9763 - val_loss: 0.1542 - val_accuracy: 0.9720
Epoch 4/5
250/250 [==============================] - 25s 100ms/step - loss: 0.0839 - accuracy: 0.9766 - val_loss: 0.0969 - val_accuracy: 0.9825
Epoch 5/5
250/250 [==============================] - 7s 29ms/step - loss: 0.0780 - accuracy: 0.9820 - val_loss: 0.1026 - val_accuracy: 0.9850


In [6]:
# Evaluation

# Load the best model
model = tf.keras.models.load_model("model_checkpoint.h5")

# Evaluate
metrics = model.evaluate(X_test, y_test)

print(f"Test loss: {metrics[0]}")
print(f"Test accuracy: {metrics[1]}")

63/63 [==============================] - 1s 7ms/step - loss: 0.0969 - accuracy: 0.9825
Test loss: 0.09687189757823944
Test accuracy: 0.9825000166893005
